**CIFAR-10** dataset with 10 different classes, having 60000  32 * 32 colured images with 6000 images for each class.

50000 training images + 10000 testing images

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt

In [11]:
# device 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
# hyperparameter
num_epochs = 4
batch_size = 4
lr = 0.001

In [13]:
# dataset has PIL Image of range [0,1]
# we transform them to Tensors of normalzer range [-1,1]

transforms = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

train_data = torchvision.datasets.CIFAR10(root = './data',train=True,
                                          transform=transforms,download =True)

test_data = torchvision.datasets.CIFAR10(root = './data',train=False,
                                          transform=transforms,download =True)

train_loader = torch.utils.data.DataLoader(dataset = train_data,batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset = test_data,batch_size=batch_size,
                                           shuffle=False)

classes = ('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')


Files already downloaded and verified
Files already downloaded and verified


In [23]:
# model (implement conv net)
class ConvNet(nn.Module):
  def __init__(self):
    super(ConvNet,self).__init__()
    # torch.Size([4,3,32,32])
    self.conv1 = nn.Conv2d(3,6,5) 
    # torch.Size([4,6,28,28])
    self.pool = nn.MaxPool2d(2,2) 
    # torch.Size([4,6,14,14])
    self.conv2 = nn.Conv2d(6,16,5) 
    # torch.Size([4,16,10,10])
    self.fc1 = nn.Linear(16*5*5,120)  # input size after cov + pool = 16*5*5
    self.fc2 = nn.Linear(120,84)
    self.fc3 = nn.Linear(84,10) # num_classes = 10

  def forward(self,x):
    out = self.pool(F.relu(self.conv1(x)))
    out = self.pool(F.relu(self.conv2(out)))
    out = out.view(-1,16*5*5)
    out = F.relu(self.fc1(out))
    out = F.relu(self.fc2(out))
    out = self.fc3(out)
    return out

model = ConvNet().to(device)

# loss and optimizer
criterion = nn.CrossEntropyLoss() # softmax included
optimizer = torch.optim.SGD(model.parameters(),lr = lr)

# training
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
  for i,(images,labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)

    # forward pass
    outputs = model(images)
    loss = criterion(outputs,labels)

    # backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1) % 2000 == 0:
      print(f'Epoch [{epoch+1}/{num_epochs}],Step [{i+1}/{n_total_steps}],Loss:{loss.item():.4f}')

print('Finished training!')

# test
with torch.no_grad():
  n_correct = 0
  n_samples = 0
  n_class_correct = [0 for i in range(10)]
  n_class_samples = [0 for i in range(10)]
  for images,labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    # max returns (value,index)
    _,predicted = torch.max(outputs,1)
    n_samples += labels.size(0)
    n_correct += (predicted == labels).sum().item()

    for i in range(batch_size):
      label = labels[i]
      pred = predicted[i]
      if (label == pred):
        n_class_correct[label] += 1
      n_class_samples[label] += 1

  acc = 100.0 * n_correct/n_samples
  print(f' Accuracy of NN: {acc} %')

  for i in range(10):
      acc = 100.0 * n_class_correct[i] / n_class_samples[i]
      print(f'Accuracy of {classes[i]}: {acc} %')



Epoch [1/4],Step [2000/12500],Loss:2.3187
Epoch [1/4],Step [4000/12500],Loss:2.3138
Epoch [1/4],Step [6000/12500],Loss:2.3139
Epoch [1/4],Step [8000/12500],Loss:2.2996
Epoch [1/4],Step [10000/12500],Loss:2.2806
Epoch [1/4],Step [12000/12500],Loss:1.7359
Epoch [2/4],Step [2000/12500],Loss:2.0335
Epoch [2/4],Step [4000/12500],Loss:1.8260
Epoch [2/4],Step [6000/12500],Loss:1.2505
Epoch [2/4],Step [8000/12500],Loss:3.2502
Epoch [2/4],Step [10000/12500],Loss:2.0098
Epoch [2/4],Step [12000/12500],Loss:1.8174
Epoch [3/4],Step [2000/12500],Loss:1.6380
Epoch [3/4],Step [4000/12500],Loss:1.5248
Epoch [3/4],Step [6000/12500],Loss:1.6581
Epoch [3/4],Step [8000/12500],Loss:1.4933
Epoch [3/4],Step [10000/12500],Loss:1.3581
Epoch [3/4],Step [12000/12500],Loss:1.1961
Epoch [4/4],Step [2000/12500],Loss:1.9643
Epoch [4/4],Step [4000/12500],Loss:1.7751
Epoch [4/4],Step [6000/12500],Loss:2.2162
Epoch [4/4],Step [8000/12500],Loss:1.4095
Epoch [4/4],Step [10000/12500],Loss:1.5980
Epoch [4/4],Step [12000/125